## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# Define Schema and type according to uses
from pyspark.sql.types import StructField, StructType, DateType, DoubleType, IntegerType

df_schema = StructType([
  StructField("Date", DateType(), nullable=True),
  StructField("Open", DoubleType(), nullable=True),
  StructField("High", DoubleType(), nullable=True),
  StructField("Low", DoubleType(), nullable=True),
  StructField("Close", DoubleType(), nullable=True),
  StructField("Adj_Close", DoubleType(), nullable=True),
  StructField("Volume", IntegerType(), nullable=True),
])

In [0]:
# File location and type
file_location = "/FileStore/tables/AAPL-1.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .schema(df_schema) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

# display(df)
# print(df.count()) === 10197

In [0]:
#  Display Dataframe's Schema
df.printSchema()

root
-- Date: date (nullable = true)
-- Open: double (nullable = true)
-- High: double (nullable = true)
-- Low: double (nullable = true)
-- Close: double (nullable = true)
-- Adj_Close: double (nullable = true)
-- Volume: integer (nullable = true)

In [0]:
#  filter transformation
close_less_than_500 = df.filter("Close < 500").show()
# print(close_less_than_500) === 10197

+----------+--------+--------+--------+--------+---------+---------+
 Date| Open| High| Low| Close|Adj_Close| Volume|
+----------+--------+--------+--------+--------+---------+---------+
1980-12-12|0.128348|0.128906|0.128348|0.128348| 0.100751|469033600|
1980-12-15| 0.12221| 0.12221|0.121652|0.121652| 0.095495|175884800|
1980-12-16|0.113281|0.113281|0.112723|0.112723| 0.088485|105728000|
1980-12-17|0.115513|0.116071|0.115513|0.115513| 0.090676| 86441600|
1980-12-18|0.118862| 0.11942|0.118862|0.118862| 0.093304| 73449600|
1980-12-19|0.126116|0.126674|0.126116|0.126116| 0.098999| 48630400|
1980-12-22|0.132254|0.132813|0.132254|0.132254| 0.103817| 37363200|
1980-12-23|0.137835|0.138393|0.137835|0.137835| 0.108198| 46950400|
1980-12-24|0.145089|0.145647|0.145089|0.145089| 0.113892| 48003200|
1980-12-26|0.158482| 0.15904|0.158482|0.158482| 0.124405| 55574400|
1980-12-29|0.160714|0.161272|0.160714|0.160714| 0.126158| 93161600|
1980-12-30|0.157366|0.157366|0.156808|0.156808| 0.123091| 68880000|
1980-12-31|0.152902|0.152902|0.152344|0.152344| 0.119587| 35750400|
1981-01-02|0.154018|0.155134|0.154018|0.154018| 0.120901| 21660800|
1981-01-05|0.151228|0.151228| 0.15067| 0.15067| 0.118273| 35728000|
1981-01-06|0.144531|0.144531|0.143973|0.143973| 0.113016| 45158400|
1981-01-07|0.138393|0.138393|0.137835|0.137835| 0.108198| 55686400|
1981-01-08|0.135603|0.135603|0.135045|0.135045| 0.106008| 39827200|
1981-01-09|0.142299|0.142857|0.142299|0.142299| 0.111702| 21504000|
1981-01-12|0.142299|0.142299|0.141183|0.141183| 0.110826| 23699200|
+----------+--------+--------+--------+--------+---------+---------+
only showing top 20 rows

In [0]:
#  filter data by columns

df.filter("Close < 500").select(["Open", "Close"]).show()

+--------+--------+
 Open| Close|
+--------+--------+
0.128348|0.128348|
 0.12221|0.121652|
0.113281|0.112723|
0.115513|0.115513|
0.118862|0.118862|
0.126116|0.126116|
0.132254|0.132254|
0.137835|0.137835|
0.145089|0.145089|
0.158482|0.158482|
0.160714|0.160714|
0.157366|0.156808|
0.152902|0.152344|
0.154018|0.154018|
0.151228| 0.15067|
0.144531|0.143973|
0.138393|0.137835|
0.135603|0.135045|
0.142299|0.142299|
0.142299|0.141183|
+--------+--------+
only showing top 20 rows

In [0]:
#  filter by one column and show other
df.filter(df["Close"] < 500).select("Volume").show()

+---------+
 Volume|
+---------+
469033600|
175884800|
105728000|
 86441600|
 73449600|
 48630400|
 37363200|
 46950400|
 48003200|
 55574400|
 93161600|
 68880000|
 35750400|
 21660800|
 35728000|
 45158400|
 55686400|
 39827200|
 21504000|
 23699200|
+---------+
only showing top 20 rows

In [0]:
#  filter data on multiple condition closing price < $200 and opening price > $200

df.filter((df["Close"] < 0.12 ) & (df["Open"] > 0.12)).show()

+----------+--------+--------+--------+--------+---------+---------+
 Date| Open| High| Low| Close|Adj_Close| Volume|
+----------+--------+--------+--------+--------+---------+---------+
1981-05-14|0.121094|0.121094|0.119978|0.119978| 0.094181| 4928000|
1984-01-09|0.123884|0.123884|0.113281|0.117188| 0.09199|215734400|
1984-01-27|0.123326|0.123884|0.114397|0.116629| 0.091552|194096000|
1984-02-23| 0.12221| 0.12221|0.116071|0.119978| 0.094181|155052800|
1984-02-28|0.120536|0.121094|0.112165|0.113839| 0.089362|169926400|
1984-03-05|0.121652| 0.12221|0.117746| 0.11942| 0.093742| 73606400|
1984-03-14|0.120536|0.121094|0.118304|0.118862| 0.093304| 59606400|
1984-06-26|0.121652| 0.12221|0.116071|0.116071| 0.091114|148646400|
1984-07-30|0.121094|0.121652|0.112723|0.113839| 0.089362|125036800|
1984-08-31|0.120536|0.121094|0.116629|0.118304| 0.092866|137849600|
1984-09-21|0.121094|0.124442|0.118304|0.119978| 0.094181| 99836800|
1984-12-10|0.121652|0.121652| 0.11942| 0.11942| 0.093742|111484800|
1985-02-20|0.123326|0.123884|0.117746|0.117746| 0.092428|219968000|
1985-02-26|0.121652| 0.12221| 0.11942| 0.11942| 0.093742|188966400|
1986-03-24|0.123326|0.123326|0.117746| 0.11942| 0.093742|294313600|
1986-04-04|0.120536|0.120536|0.118862| 0.11942| 0.093742|125955200|
1986-04-14|0.120536|0.121652| 0.11942|0.119978| 0.094181| 84963200|
1997-07-01|0.124442| 0.125|0.117188|0.117746| 0.10125|450676800|
1997-12-22|0.123884| 0.125|0.117746|0.118862| 0.10221|159476800|
+----------+--------+--------+--------+--------+---------+---------+

In [0]:
# Import Relevant Libraries
from pyspark.sql.functions import  year,first, last

#  data timeperiod
start_year = year(first(df["Date"]))
end_year = year(last(df["Date"]))

data_duration = end_year - start_year
print(data_duration)

#  Data description 
df.describe().show()

Column<'(year(last(Date)) - year(first(Date)))'>
+-------+------------------+-----------------+-----------------+------------------+-----------------+--------------------+
summary| Open| High| Low| Close| Adj_Close| Volume|
+-------+------------------+-----------------+-----------------+------------------+-----------------+--------------------+
 count| 10197| 10197| 10197| 10197| 10197| 10162|
 mean|11.061229938119068|11.18024408433853|10.93796137530646|11.062961948906544|10.48474150828673|3.2892767753395003E8|
 stddev|22.629175079172278|22.88693314591072|22.35312908554958| 22.6281187313256|22.29778606321341| 3.054240958808207E8|
 min| 0.049665| 0.049665| 0.049107| 0.049107| 0.038548| 0|
 max| 143.600006| 145.089996| 141.369995| 143.160004| 142.70401| 2127171200|
+-------+------------------+-----------------+-----------------+------------------+-----------------+--------------------+

In [0]:
#  Add and Rename Colums

df.withColumn("Double_Adj_Close", df["Adj_Close"]*2).select("Adj_Close", "Double_Adj_Close").show()

+---------+----------------+
Adj_Close|Double_Adj_Close|
+---------+----------------+
 0.100751| 0.201502|
 0.095495| 0.19099|
 0.088485| 0.17697|
 0.090676| 0.181352|
 0.093304| 0.186608|
 0.098999| 0.197998|
 0.103817| 0.207634|
 0.108198| 0.216396|
 0.113892| 0.227784|
 0.124405| 0.24881|
 0.126158| 0.252316|
 0.123091| 0.246182|
 0.119587| 0.239174|
 0.120901| 0.241802|
 0.118273| 0.236546|
 0.113016| 0.226032|
 0.108198| 0.216396|
 0.106008| 0.212016|
 0.111702| 0.223404|
 0.110826| 0.221652|
+---------+----------------+
only showing top 20 rows

In [0]:
df.withColumnRenamed("Adj_Close", "Adjusted_Close_Price").show()

+----------+--------+--------+--------+--------+--------------------+---------+
 Date| Open| High| Low| Close|Adjusted_Close_Price| Volume|
+----------+--------+--------+--------+--------+--------------------+---------+
1980-12-12|0.128348|0.128906|0.128348|0.128348| 0.100751|469033600|
1980-12-15| 0.12221| 0.12221|0.121652|0.121652| 0.095495|175884800|
1980-12-16|0.113281|0.113281|0.112723|0.112723| 0.088485|105728000|
1980-12-17|0.115513|0.116071|0.115513|0.115513| 0.090676| 86441600|
1980-12-18|0.118862| 0.11942|0.118862|0.118862| 0.093304| 73449600|
1980-12-19|0.126116|0.126674|0.126116|0.126116| 0.098999| 48630400|
1980-12-22|0.132254|0.132813|0.132254|0.132254| 0.103817| 37363200|
1980-12-23|0.137835|0.138393|0.137835|0.137835| 0.108198| 46950400|
1980-12-24|0.145089|0.145647|0.145089|0.145089| 0.113892| 48003200|
1980-12-26|0.158482| 0.15904|0.158482|0.158482| 0.124405| 55574400|
1980-12-29|0.160714|0.161272|0.160714|0.160714| 0.126158| 93161600|
1980-12-30|0.157366|0.157366|0.156808|0.156808| 0.123091| 68880000|
1980-12-31|0.152902|0.152902|0.152344|0.152344| 0.119587| 35750400|
1981-01-02|0.154018|0.155134|0.154018|0.154018| 0.120901| 21660800|
1981-01-05|0.151228|0.151228| 0.15067| 0.15067| 0.118273| 35728000|
1981-01-06|0.144531|0.144531|0.143973|0.143973| 0.113016| 45158400|
1981-01-07|0.138393|0.138393|0.137835|0.137835| 0.108198| 55686400|
1981-01-08|0.135603|0.135603|0.135045|0.135045| 0.106008| 39827200|
1981-01-09|0.142299|0.142857|0.142299|0.142299| 0.111702| 21504000|
1981-01-12|0.142299|0.142299|0.141183|0.141183| 0.110826| 23699200|
+----------+--------+--------+--------+--------+--------------------+---------+
only showing top 20 rows

In [0]:
# Grouping and Aggregateing Data
# Import Relevant Libraries
from pyspark.sql.functions import dayofmonth, hour, dayofyear, weekofyear, month, year,format_number,date_format,mean, date_format, datediff, to_date, lit

#  To Know the average closing price per year

new_df = df.withColumn("Year", year(df["Date"]))
new_df.show()

+----------+--------+--------+--------+--------+---------+---------+----+
 Date| Open| High| Low| Close|Adj_Close| Volume|Year|
+----------+--------+--------+--------+--------+---------+---------+----+
1980-12-12|0.128348|0.128906|0.128348|0.128348| 0.100751|469033600|1980|
1980-12-15| 0.12221| 0.12221|0.121652|0.121652| 0.095495|175884800|1980|
1980-12-16|0.113281|0.113281|0.112723|0.112723| 0.088485|105728000|1980|
1980-12-17|0.115513|0.116071|0.115513|0.115513| 0.090676| 86441600|1980|
1980-12-18|0.118862| 0.11942|0.118862|0.118862| 0.093304| 73449600|1980|
1980-12-19|0.126116|0.126674|0.126116|0.126116| 0.098999| 48630400|1980|
1980-12-22|0.132254|0.132813|0.132254|0.132254| 0.103817| 37363200|1980|
1980-12-23|0.137835|0.138393|0.137835|0.137835| 0.108198| 46950400|1980|
1980-12-24|0.145089|0.145647|0.145089|0.145089| 0.113892| 48003200|1980|
1980-12-26|0.158482| 0.15904|0.158482|0.158482| 0.124405| 55574400|1980|
1980-12-29|0.160714|0.161272|0.160714|0.160714| 0.126158| 93161600|1980|
1980-12-30|0.157366|0.157366|0.156808|0.156808| 0.123091| 68880000|1980|
1980-12-31|0.152902|0.152902|0.152344|0.152344| 0.119587| 35750400|1980|
1981-01-02|0.154018|0.155134|0.154018|0.154018| 0.120901| 21660800|1981|
1981-01-05|0.151228|0.151228| 0.15067| 0.15067| 0.118273| 35728000|1981|
1981-01-06|0.144531|0.144531|0.143973|0.143973| 0.113016| 45158400|1981|
1981-01-07|0.138393|0.138393|0.137835|0.137835| 0.108198| 55686400|1981|
1981-01-08|0.135603|0.135603|0.135045|0.135045| 0.106008| 39827200|1981|
1981-01-09|0.142299|0.142857|0.142299|0.142299| 0.111702| 21504000|1981|
1981-01-12|0.142299|0.142299|0.141183|0.141183| 0.110826| 23699200|1981|
+----------+--------+--------+--------+--------+---------+---------+----+
only showing top 20 rows

In [0]:
from pyspark.sql import functions as f
#  Group and aggregate data
new_df1 = new_df.groupBy("Year").agg(f.max("Close").alias("Max Close"), f.min("Close").alias("Min Close"), f.mean("Close").alias("Average Close")).orderBy("Year")

In [0]:
#  Import relevant function
from pyspark.sql.functions import format_number, col

# Select the appropiate columns to formate
cols = ['Max Close', 'Min Close', 'Average Close']
formatted_df = new_df1.select("Year", *[format_number(col(col_name),2).name(col_name) for col_name in cols])

formatted_df.show()

+----+---------+---------+-------------+
Year|Max Close|Min Close|Average Close|
+----+---------+---------+-------------+
1980| 0.16| 0.11| 0.14|
1981| 0.15| 0.06| 0.11|
1982| 0.15| 0.05| 0.09|
1983| 0.28| 0.08| 0.17|
1984| 0.15| 0.10| 0.12|
1985| 0.14| 0.06| 0.09|
1986| 0.20| 0.10| 0.14|
1987| 0.53| 0.18| 0.35|
1988| 0.42| 0.32| 0.37|
1989| 0.44| 0.30| 0.37|
1990| 0.42| 0.22| 0.34|
1991| 0.65| 0.37| 0.47|
1992| 0.62| 0.39| 0.49|
1993| 0.58| 0.20| 0.37|
1994| 0.39| 0.22| 0.30|
1995| 0.44| 0.28| 0.36|
1996| 0.31| 0.15| 0.22|
1997| 0.26| 0.12| 0.16|
1998| 0.38| 0.14| 0.27|
1999| 1.05| 0.29| 0.52|
+----+---------+---------+-------------+
only showing top 20 rows

In [0]:
#  Import relevant functions
from pyspark.sql.functions import date_format, datediff, to_date, lit, UserDefinedFunction, month
from pyspark.sql.types import StringType
from pyspark.sql import functions as f

#  Create months list
month_lst = ['January', 'Feburary', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

# Define the function
udf = UserDefinedFunction(lambda x : month_lst[int(x%12) -1], StringType())

#  Add column to df with the number of the monthsof the year
df = df.withColumn("moy_number", month(df.Date))

#  Apply function and generate a column with the name of the month of the year
df = df.withColumn("moy_name", udf("moy_number"))
df.show()

+----------+--------+--------+--------+--------+---------+---------+----------+--------+
 Date| Open| High| Low| Close|Adj_Close| Volume|moy_number|moy_name|
+----------+--------+--------+--------+--------+---------+---------+----------+--------+
1980-12-12|0.128348|0.128906|0.128348|0.128348| 0.100751|469033600| 12|December|
1980-12-15| 0.12221| 0.12221|0.121652|0.121652| 0.095495|175884800| 12|December|
1980-12-16|0.113281|0.113281|0.112723|0.112723| 0.088485|105728000| 12|December|
1980-12-17|0.115513|0.116071|0.115513|0.115513| 0.090676| 86441600| 12|December|
1980-12-18|0.118862| 0.11942|0.118862|0.118862| 0.093304| 73449600| 12|December|
1980-12-19|0.126116|0.126674|0.126116|0.126116| 0.098999| 48630400| 12|December|
1980-12-22|0.132254|0.132813|0.132254|0.132254| 0.103817| 37363200| 12|December|
1980-12-23|0.137835|0.138393|0.137835|0.137835| 0.108198| 46950400| 12|December|
1980-12-24|0.145089|0.145647|0.145089|0.145089| 0.113892| 48003200| 12|December|
1980-12-26|0.158482| 0.15904|0.158482|0.158482| 0.124405| 55574400| 12|December|
1980-12-29|0.160714|0.161272|0.160714|0.160714| 0.126158| 93161600| 12|December|
1980-12-30|0.157366|0.157366|0.156808|0.156808| 0.123091| 68880000| 12|December|
1980-12-31|0.152902|0.152902|0.152344|0.152344| 0.119587| 35750400| 12|December|
1981-01-02|0.154018|0.155134|0.154018|0.154018| 0.120901| 21660800| 1| January|
1981-01-05|0.151228|0.151228| 0.15067| 0.15067| 0.118273| 35728000| 1| January|
1981-01-06|0.144531|0.144531|0.143973|0.143973| 0.113016| 45158400| 1| January|
1981-01-07|0.138393|0.138393|0.137835|0.137835| 0.108198| 55686400| 1| January|
1981-01-08|0.135603|0.135603|0.135045|0.135045| 0.106008| 39827200| 1| January|
1981-01-09|0.142299|0.142857|0.142299|0.142299| 0.111702| 21504000| 1| January|
1981-01-12|0.142299|0.142299|0.141183|0.141183| 0.110826| 23699200| 1| January|
+----------+--------+--------+--------+--------+---------+---------+----------+--------+
only showing top 20 rows

In [0]:
sc.stop()